In [1]:
import pymysql


In [2]:
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='1234',
    # db='shoppingmall',
    charset='utf8'
)

In [3]:
cur = conn.cursor()
cur.execute('''drop schema if exists shoppingmall''')
cur.execute('''create schema shoppingmall''')
cur.execute('''use shoppingmall''')
conn.commit()

In [4]:
queryDelUserTableBeforeCreate = '''drop table if exists user_table'''
queryCreateUserTable = '''
create table user_table
(
    userID    char(8)     not null comment '사용자 ID',
    userName  varchar(10) not null comment '이름',
    birthYear int         not null comment '출생년도',
    addr      char(2)     not null comment '경기, 서울 등 2글자',
    mobile1   char(3)     null comment '휴대폰 국번',
    mobile2   char(8)     null comment '휴대폰 나머지 전화번호',
    height    smallint    null comment '키',
    mDate     DATE        null comment '회원 가입일',
    constraint pk_userID
        primary key (userID)
)
'''
queryDelPRDTableBeforeCreate = '''drop table if exists buy_table'''
queryCreateBuyTable = '''
create table buy_table
(
    num       int auto_increment comment '순번',
    userID    char(8)  not null comment '아이디 (FK)',
    prodName  char(6)  not null comment '물품명',
    groupName char(4)  null comment '분류',
    price     int      not null comment '단가',
    amount    smallint not null comment '수량',
    constraint buy_table_pk
        primary key (num),
    constraint buy_table_user_table_userID_fk
        foreign key (userID) references user_table (userID)
)
'''
cur.execute(queryDelUserTableBeforeCreate)
cur.execute(queryCreateUserTable)
cur.execute(queryDelPRDTableBeforeCreate)
cur.execute(queryCreateBuyTable)
conn.commit()

In [5]:

userID = (
'KHD',
'KJD',
'KKJ',
'KYM',
'LHJ',
'LKK',
'NHS',
'PSH',
'SDY',
'YJS',
)

userName = (
'강호동',
'김제동',
'김국진',
'김용만',
'이휘재',
'이경규',
'남희석',
'박수홍',
'신동엽',
'유재석',
)

birthYear = (
'1970',
'1974',
'1965',
'1967',
'1972',
'1960',
'1971',
'1970',
'1971',
'1972',
)

addr = (
'경북',
'경남',
'서울',
'서울',
'경기',
'경남',
'충남',
'서울',
'경기',
'서울'
)

mobile1 = (
'011',
None,
'019',
'010',
'011',
'018',
'016',
'010',
None,
'010',
)

mobile2 = (
'22222222',
None,
'33333333',
'44444444',
'88888888',
'99999999',
'66666666',
'00000000',
None,
'11111111',
)

height = (
'182',
'173',
'171',
'177',
'180',
'170',
'180',
'183',
'176',
'178',
)

mDate = (
'2007-07-07',
'2013-03-03',
'2009-09-09',
'2015-05-05',
'2006-04-04',
'2004-12-12',
'2017-04-04',
'2012-05-05',
'2008-10-10',
'2008-08-08',
)


In [6]:
aggregatedUser = list(zip(userID, userName, birthYear, addr, mobile1, mobile2, height, mDate))

In [7]:
queryInsertUserTable = '''
insert into user_table (userID, userName, birthYear, addr, mobile1, mobile2, height, mDate)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
'''
cur.executemany(queryInsertUserTable, aggregatedUser)
conn.commit()

In [8]:
num = [None for x in range(12)]
userID = [
'KHD',
'KHD',
'KYM',
'PSH',
'KHD',
'PSH',
'KJD',
'LHJ',
'LHJ',
'PSH',
'LHJ',
'PSH',
]

prodName = [
'운동화',
'노트북',
'모니터',
'모니터',
'청바지',
'메모리',
'책',
'책',
'청바지',
'운동화',
'책',
'운동화',
]

groupName = [
None,
'전자',
'전자',
'전자',
'의류',
'전자',
'서적',
'서적',
'의류',
None,
'서적',
None,
]

price = [
30,
1000,
200,
200,
50,
80,
15,
15,
50,
30,
15,
30,
]

amount = [
2,
1,
1,
5,
3,
10,
5,
2,
1,
2,
1,
2,
]

In [9]:
aggregatedBuy = list(zip(num, userID, prodName, groupName, price, amount))

In [10]:
queryInsertBuyTable = '''
insert into buy_table (num, userID, prodName, groupName, price, amount)
VALUES (%s,%s,%s,%s,%s,%s)
'''
cur.executemany(queryInsertBuyTable, aggregatedBuy)
conn.commit()

In [11]:
def answerPrint(ansNum = 1, query='',conn = conn, cur = cur):
    cur.execute(query)
    result = cur.fetchall()
    print(f"문제 {ansNum}번")
    print("-------------------------------------------")
    print("-------------------------------------------")

In [12]:
def ansPrint(ansNum, query, cur, conn):
    cur.execute(query)
    header = [x[0] for x in cur.description]
    headertxt = '  '.join(header)
    result = cur.fetchall()
    print(f"문제 {ansNum}번")
    print("-" * (len(headertxt) + 12))
    print(f"{headertxt}")
    print("-" * (len(headertxt) + 12))
    for row in result:
        print(row)

In [13]:
# 문제 1번 결과
query1 = '''select userName, prodName, addr, concat(mobile1, mobile2) as 연락처
from user_table inner join buy_table
on buy_table.userID = user_table.userID;'''

# 문제2번 결과
query2 = '''select u.userID,
       u.userName,
       buy_table.prodName,
       u.addr,
       concat(u.mobile1, u.mobile2) as 연락처
from user_table as u inner join buy_table
    on buy_table.userID = u.userID
where u.userID = 'KYM';'''

#문제3
query3 = '''select u.userID, u.userName, prodName, addr, concat(mobile1, mobile2) as 연락처
from shoppingmall.user_table as u inner join shoppingmall.buy_table bt
    on u.userID = bt.userID
order by userID asc;'''

#문제4
query4 = '''select distinct ut.userID, userName, addr
from buy_table as bt inner join shoppingmall.user_table ut
    on bt.userID = ut.userID
order by userID asc;'''

#문제5
query5 = '''select ut.userID, ut.userName, ut.addr, concat(mobile1, mobile2) as 연락처
from buy_table as bt inner join shoppingmall.user_table ut
    on bt.userID = ut.userID
where addr in ('경북', '경남')
;'''

querylist = [query1, query2, query3, query4, query5]

for idx, query in enumerate(querylist, 1):
    ansPrint(idx, query, cur, conn)

문제 1번
-----------------------------------------
userName  prodName  addr  연락처
-----------------------------------------
('강호동', '운동화', '경북', '01122222222')
('강호동', '노트북', '경북', '01122222222')
('강호동', '청바지', '경북', '01122222222')
('김제동', '책', '경남', None)
('김용만', '모니터', '서울', '01044444444')
('이휘재', '책', '경기', '01188888888')
('이휘재', '청바지', '경기', '01188888888')
('이휘재', '책', '경기', '01188888888')
('박수홍', '모니터', '서울', '01000000000')
('박수홍', '메모리', '서울', '01000000000')
('박수홍', '운동화', '서울', '01000000000')
('박수홍', '운동화', '서울', '01000000000')
문제 2번
-------------------------------------------------
userID  userName  prodName  addr  연락처
-------------------------------------------------
('KYM', '김용만', '모니터', '서울', '01044444444')
문제 3번
-------------------------------------------------
userID  userName  prodName  addr  연락처
-------------------------------------------------
('KHD', '강호동', '운동화', '경북', '01122222222')
('KHD', '강호동', '노트북', '경북', '01122222222')
('KHD', '강호동', '청바지', '경북', '01122222222')
('K

In [14]:
cur.close()
conn.close()